# Transition period for EIP 1559

###### October 2020

---

In this notebook we model and simulate the transition period suggestested in [Pull Request 2924](https://github.com/ethereum/EIPs/pull/2924). To simulate the transition period we will use a few classes from our [`abm1559`](https://github.com/barnabemonnot/abm1559) library, as well as the Strategic User presented in the [previous notebook](https://github.com/barnabemonnot/abm1559/blob/master/notebooks/stationary1559.ipynb).

In [ ]:
import os, sys
sys.path.insert(1, os.path.realpath(os.path.pardir))
# You may remove the two lines above if you have installed abm1559 from pypi

from typing import Sequence

from abm1559.utils import (
    constants,
    get_basefee_bounds,
    flatten
)

from abm1559.txpool import TxPool

from abm1559.users import (
    User1559,
    AffineUser
)

from abm1559.txs import Transaction, Tx1559

from abm1559.userpool import UserPool

from abm1559.chain import (
    Chain,
    Block1559,
    Block
)

from abm1559.simulator import (
    spawn_poisson_heterogeneous_demand,
    update_basefee,
)

import pandas as pd
import numpy as np

In [ ]:
class OptimisticUser(User1559):
    def expected_time(self, params):
        return 1

class StrategicUser(User1559):
    """
    A strategic affine user sending 1559 transactions.

    - Expects to be included in the next block
    - Prefers not to participate if its expected payoff is negative
    - Strategic gas_premium
    """

    epsilon = 0.1 # how much the user overbids by

    def expected_time(self, params):
        return 1

    def decide_parameters(self, params):
        if params["min_premium"] is None:
            min_premium = 1 * (10 ** 9)
        else:
            min_premium = params["min_premium"]

        gas_premium = min_premium + self.epsilon * (10 ** 9)
        max_fee = self.value

        return {
            "max_fee": max_fee, # in wei
            "gas_premium": gas_premium, # in wei
            "start_block": self.wakeup_block
        }

    def export(self):
        return {
            **super().export(),
            "user_type": "strategic_user_1559",
        }

    def __str__(self):
        return f"1559 strategic affine user with value {self.value} and cost {self.cost_per_unit}"


We assume that during the transitional period, part of the ecosystem will remain using legacy tooling and creating transactions of the legacy format. There is a question of how this legacy tooling that isn't upgraded to have visibility and use `1559` will interact with the overall system. 

To deal with that issue the Pull Request suggests that Legacy Transactions get normalized into `1559` transactions using a simple normalization function. There is the open question of how the legacy APIs will implement and normalize the fields. For the simulations we assume an inverse of the `normalize_from_legacy` is used, where the `max_fee` and `gas_premium` variables are just treated as `gas_price` to keep interoperability during the transition period.

In [ ]:
class TxLegacy(Transaction):
    """
    Inherits from :py:class:`abm1559.txs.Transaction`. A legacy-type transaction.
    """
    def __init__(self, sender, params, gas_used = constants["SIMPLE_TRANSACTION_GAS"]):
        super().__init__(sender, params, gas_used = gas_used)

        self.gas_price = params["gas_price"]
        self.gas_limit = params["gas_limit"]

    def __str__(self):
        return f"Legacy Transaction {self.tx_hash.hex()}: gas_limit {self.gas_limit}, gas_price {self.gas_price}"

    def tx_data(self, params):
        return {
            **super().tx_data(params),
            "gas_limit": self.gas_limit / (10 ** 9),
            "gas_price": self.gas_price / (10 ** 9),
        }

def normalize_from_legacy(legacy: TxLegacy) -> Tx1559:
    """
    Normalized according to
    https://github.com/ethereum/EIPs/blob/541c8be92fe759aa602b7d06a088ab1a139e37ce/EIPS/eip-1559.md
    """
    normalized_params = {
        "gas_limit": legacy.gas_limit,
        "max_fee": legacy.gas_price,
        "gas_premium": legacy.gas_price,
        "start_block": legacy.start_block,
    }
    return Tx1559(
        sender=legacy.sender,
        params=normalized_params
    )

def normalize_to_legacy(input_tx: Tx1559) -> TxLegacy:
    """
    Normalises the environment variables so legacy tooling is able to interact with 1559
    """
    
    normalized_params = {
        "gas_price": input_tx.gas_premium,
        "gas_limit": input_tx.gas_used,
        "start_block": input_tx.start_block,
    }
    return TxLegacy(
        sender=input_tx.sender,
        params=normalized_params
    )
    

Next we introduce Legacy agents with strategic behaviours that are analogous to different gas price estimation methods currently being used.

---

We start with the Web3Js library, which sets gas prices for transactions on the [web3.eth.gasPrice](https://github.com/ethereum/web3.js/blob/79a4ce4e330605d0b7a508899f53c759d701c197/DOCUMENTATION.md#web3ethgasprice).
Agents that use Web3Js for gas prediction, set their prices as the median of the price over the last `n` blocks. User bids this, if no tx data is available, it uses the minimum gas price as seen in the environment (this is a workaround for when there are no blocks present in the history - as that sets an initial bid set as the minimum `gas_premium`)

One last issue stands before we are able to simulate the legacy market together with the new proposed 1559 mechanism. How will the legacy users understand the new environment? Here we suggest a naive normalization function that translates how the market can be translated to work with the new mechanism.

In [ ]:
def normalize_environment(params):
    normalized_blocks = []
    if params["recent_blocks"]:
        for block in params["recent_blocks"]:
            if block.txs:
                normalized_txs = [normalize_to_legacy(tx) for tx in block.txs]
                normalized_block = Block(normalized_txs, block.parent_hash, block.height)
                normalized_blocks.append(normalized_block)
            else:
                continue
    
    normalized_env = {
        # legacy user doesn't know what basefee is
        # "basefee": constants["INITIAL_BASEFEE"],    
        "current_block": None,
        # min premium is understood as min_gas_price
        "min_price": params["min_premium"],
        "recent_blocks": normalized_blocks,
    }
    return normalized_env

In [ ]:
class LegacyWeb3jsUser(AffineUser):
    def expected_time(self, params):
        return 5

    def decide_parameters(self, params):
        all_txs = [block.txs for block in params["recent_blocks"]]
        fees = [tx.gas_price for tx in flatten(all_txs)]
        gas_price = np.median(fees) if len(fees) > 0 else params["min_price"]
        gas_limit = constants["SIMPLE_TRANSACTION_GAS"]
        return {
            "gas_limit": gas_limit, # in wei
            "gas_used": gas_limit, # in wei
            "gas_price": gas_price, # in wei
            "start_block": self.wakeup_block,
        }
    
    # This function is the main user entry point 
    def create_transaction(self, params: dict):
        # This user interacts with the environment params but only understands it pre-1559
        normalized_params = normalize_environment(params)
        tx_params = self.decide_parameters(normalized_params)

        expected_waiting_time = self.expected_time(params = {})
        expected_gas_price = tx_params["gas_price"]
        expected_block = self.wakeup_block + expected_waiting_time

        expected_payoff = self.payoff({
            "gas_price": expected_gas_price,
            "current_block": expected_block,
        })

        if expected_payoff < 0:
            return None

        tx = TxLegacy(
            sender = self.pub_key,
            params = tx_params,
        )
        return tx

We also include an agent that creates Legacy transactions with the expectation of 5 block wait time

In [6]:
class LegacyUser(AffineUser):
    """
    An affine user sending Legacy transactions.
    """
    # Expects to be included within 5 blocks
    # Prefers not to participate if its expected payoff is negative
    # Fixed gas_premium

    def expected_time(self, params):
        return 5

    def decide_parameters(self, params):
        gas_price = self.value - self.expected_time(params) * self.cost_per_unit
        gas_limit = constants["SIMPLE_TRANSACTION_GAS"]
        return {
            "gas_limit": gas_limit, # in wei
            "gas_used": gas_limit, # in wei
            "gas_price": gas_price, # in wei
            "start_block": self.wakeup_block,
        }

    def create_transaction(self, params: dict):
        tx_params = self.decide_parameters(params)

        expected_waiting_time = self.expected_time(params = {})
        basefee_lower_bound = get_basefee_bounds(params["basefee"], expected_waiting_time)["lb"]
        expected_gas_price = tx_params["gas_price"]

        # If user doesn't expect basefee to come down enough in the best case,
        # don't transact
        if basefee_lower_bound > expected_gas_price:
            return None

        expected_block = self.wakeup_block + expected_waiting_time
        expected_payoff = self.payoff({
            "gas_price": expected_gas_price,
            "current_block": expected_block,
        })

        if expected_payoff < 0:
            return None

        tx = TxLegacy(
            sender = self.pub_key,
            params = tx_params,
        )
        return tx

    def cancel(self, tx, params):
        if "cancel_cost" in params:
            cancel_cost = params["cancel_cost"]
        else:
            cancel_cost = 0
        return self.current_value(params) - cancel_cost < 0

    def export(self):
        return {
            **super().export(),
            "user_type": "user_legacy",
        }

    def __str__(self):
        return f"Legacy affine user with value {self.value} and cost {self.cost_per_unit}"

New transaction pool that normalizes transactions.
We use the same simulation loop as before, but we allow agents to have visibility on recent blocks through the `params` variable.

In [7]:
class MixedPool(TxPool):
    def add_txs(self, txs: Sequence[Transaction]) -> None:
        for tx in txs:
            if type(tx) is TxLegacy:
                tx = normalize_from_legacy(tx)
            self.txs[tx.tx_hash] = tx
        self.pool_length += len(txs)

In [8]:
def simulate(demand_scenario, shares_scenario):
    # Instantiate a couple of things
    txpool = MixedPool()
    chain = Chain()
    metrics = []
    user_pool = UserPool()
    
    # `params` are the "environment" of the simulation
    params = {
        "basefee": constants["INITIAL_BASEFEE"],
        "current_block": None,
        "min_premium": 1 * (10 ** 9),
        "recent_blocks": [],
    }

    for t in range(len(demand_scenario)):
        # set current block wi
        params["current_block"] = t

        # We return some demand which on expectation yields 2000 new users per round
        users = spawn_poisson_heterogeneous_demand(t, demand_scenario[t], shares_scenario[t])

        # Add new users to the pool
        # We query each new user with the current basefee value
        # Users either return a transaction or None if they prefer to balk
        decided_txs = user_pool.decide_transactions(users, params)

        # New transactions are added to the transaction pool
        txpool.add_txs(decided_txs)

        # The best valid transactions are taken out of the pool for inclusion
        selected_txs = txpool.select_transactions(params)
        txpool.remove_txs([tx.tx_hash for tx in selected_txs])

        # We create a block with these transactions
        block = Block1559(txs = selected_txs, parent_hash = chain.current_head, height = t, basefee = params["basefee"])
        params["recent_blocks"].append(block)
        # Record the min premium in the block
        params["min_premium"] = block.min_premium()

        # The block is added to the chain
        chain.add_block(block)

        # A couple of metrics we will use to monitor the simulation
        # A couple of metrics we will use to monitor the simulation
        pool_strat_users = len(
                [tx for tx in txpool.txs.values() if type(user_pool.users[tx.sender]) == StrategicUser])
        pool_nonstrat_users = len(
                [tx for tx in txpool.txs.values() if type(user_pool.users[tx.sender]) == OptimisticUser])
        pool_legacy_users = len(
                [tx for tx in txpool.txs.values() if type(user_pool.users[tx.sender]) == LegacyUser])

        row_metrics = {
            "block": t,
            "basefee": params["basefee"] / (10 ** 9),
            "users": len(users),
            "strategic": len([user for user in users if type(user) is StrategicUser]),
            "nonstategic": len([user for user in users if type(user) is OptimisticUser]),
            "legacy": len([user for user in users if type(user) is LegacyUser]),
            "decided_txs": len(decided_txs),
            "included_txs": len(selected_txs),
            "blk_min_premium": block.min_premium() / (10 ** 9), # to Gwei
            "pool_length": txpool.pool_length,
            "pool_strat_users": pool_strat_users,
            "pool_nonstrat_users": pool_nonstrat_users,
            "pool_legacy_users": pool_legacy_users,
        }
        metrics.append(row_metrics)

        # Finally, basefee is updated and a new round starts
        params["basefee"] = update_basefee(block, params["basefee"])

    return (pd.DataFrame(metrics), user_pool, chain)


In [ ]:
blocks = 50
demand_scenario = [2500 for i in range(blocks)]

shares_scenario = [{
    # StrategicUser: 0.25,
    # OptimisticUser: 0.25,
#     LegacyUser: 1,
    LegacyWeb3jsUser: 1,
} for i in range(blocks)]

(df, user_pool, chain) = simulate(demand_scenario, shares_scenario)

In [ ]:
df.plot("block", ["basefee"])